In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
#be sure tf 2.0 
!pip install tensorflow==2.0.0 

In [0]:
tf.__version__

In [0]:
train_ds_url = "http://download.tensorflow.org/data/iris_training.csv"
test_ds_url = "http://download.tensorflow.org/data/iris_test.csv"
ds_columns = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth', 'Plants']
species = np.array(['Setosa', 'Versicolor', 'Virginica'], dtype=np.object)

#Load data
categories = 'Plants'

train_path = tf.keras.utils.get_file(train_ds_url.split('/')[-1], train_ds_url)
test_path = tf.keras.utils.get_file(test_ds_url.split('/')[-1], test_ds_url)

train = pd.read_csv(train_path, names=ds_columns, header=0)
train_plantfeatures, train_categories = train, train.pop(categories)

test = pd.read_csv(test_path, names=ds_columns, header=0)
test_plantfeatures, test_categories = test, test.pop(categories)

y_categorical = tf.keras.utils.to_categorical(train_categories, num_classes=3)
y_categorical_test = tf.keras.utils.to_categorical(test_categories, num_classes=3)

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((train_plantfeatures.values, y_categorical))
dataset = dataset.batch(32)
dataset = dataset.shuffle(1000)
dataset = dataset.repeat()

dataset_test = tf.data.Dataset.from_tensor_slices((test_plantfeatures.values, y_categorical_test))
dataset_test = dataset_test.batch(32)
dataset_test = dataset_test.shuffle(1000)
dataset_test = dataset_test.repeat()


#build model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(16, input_dim=4),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


In [0]:
#train model

model.fit(dataset, steps_per_epoch=32, epochs=100, verbose=1)


#eval model
loss, accuracy = model.evaluate(dataset_test, steps=32)

print("loss:%f"% (loss))
print("accuracy: %f"%   (accuracy))



In [0]:
# predict
new_specie = np.array([7.9,3.8,6.4,2.0])
predition = np.around(model.predict(np.expand_dims(new_specie, axis=0))).astype(np.int)[0]
print(model.predict(np.expand_dims(new_specie, axis=0)))
print("This species should be %s" % species[predition.astype(np.bool)][0])

model.predict(np.expand_dims(new_specie, axis=0))

# traditional saving model [option 1]
#model.save('iris_model.h5')




In [0]:
#converting to tflite from keras
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open ("iris_lite.tflite" , "wb") .write(tflite_model)

In [0]:
!ls

In [0]:
!wget https://dl.google.com/coral/python/tflite_runtime-1.14.0-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip3 install tflite_runtime-1.14.0-cp36-cp36m-linux_x86_64.whl

In [0]:
import numpy as np
import tflite_runtime.interpreter as tflite


interpreter = tflite.Interpreter(model_path="iris_lite.tflite") # change the tflite model
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model
new_specie = np.array([7.9,3.8,6.4,2.0]) # general example to predict
input_data = np.array(np.expand_dims(new_specie, axis=0), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)